In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
%autosave 20

Autosaving every 20 seconds


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

/home/jd/miniconda3/envs/eva/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from torchsummary import summary
from rf_calc import receptive_field

In [4]:
import torchvision
from torchvision import datasets,transforms

In [5]:
import warnings
warnings.filterwarnings('ignore')

import gc
import torch
import os
from tqdm import tqdm
import numpy as np

In [6]:
import os
os.getcwd()

'/home/jd/Desktop/Computer vision/Extensive_CV/Neural Architecture/Others'

In [7]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [8]:
def clean_gpu():
    """Considering we have GPUs"""

    gc.collect()
    torch.cuda.empty_cache()
    return None
    
def gpu_info():
    gpu_info = os.system("nvidia-smi")
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
        print('Not connected to a GPU')
    else:
        print(gpu_info)

In [9]:
clean_gpu()

In [10]:
use_cuda = torch.cuda.is_available() #chjecking if cuda is available or not
device = torch.device("cuda" if use_cuda else "cpu") #if gpu is available then device = cuda else cpu
torch.manual_seed(1)
batch_size = 128
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [11]:
transform_type = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize(mean=(0.1307),std=(0.3081))])

In [12]:
# Load the MNIST dataset
mnist_train = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('/home/jd/Desktop/Computer vision/Extensive_CV/data/',transform=transform_type, train=True, download=True),
    batch_size=batch_size, shuffle=True,**kwargs
)

mnist_test = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('/home/jd/Desktop/Computer vision/Extensive_CV/data/',transform=transform_type, train=False, download=True),
    batch_size=batch_size, shuffle=True,**kwargs
)

In [ ]:
nn.Ba

In [28]:
# Define the model
class MnistModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3,3), stride=1, padding=0,bias=True),
            nn.BatchNorm2d(8),
            nn.ReLU())  #op = 26 * 26

        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=1, padding=0,bias=True),
            nn.BatchNorm2d(16),
            nn.ReLU())#op = 24 * 24
        
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(3,3), stride=1, padding=0,bias=True),
            nn.BatchNorm2d(16),
            nn.ReLU()) #op = 22 * 22
        
        self.mp1 = nn.MaxPool2d(kernel_size=2,stride=2) #op = 11 * 11
        
        self.conv4 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3,3), stride=1, padding=1,bias=True),
            nn.BatchNorm2d(32),
            nn.ReLU())  #op = 11 * 11
        
        self.conv5 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3,3), stride=1, padding=0,bias=True),
            nn.BatchNorm2d(64),
            nn.ReLU())  #op = 9 * 9
        
        self.conv6 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3,3), stride=1, padding=0,bias=True),
            nn.BatchNorm2d(64),
            nn.ReLU())  #op = 7 * 7
        
        self.mp2 = nn.MaxPool2d(kernel_size=2,stride=2) #op = 4 * 4
        
        self.conv7 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3,3), stride=1, padding=1,bias=True),
            nn.BatchNorm2d(128),
            nn.ReLU() ) #op = 4 * 4
        
        self.conv8 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3,3), stride=1, padding=1,bias=True),
            nn.BatchNorm2d(256),
            nn.ReLU()) #op = 2 * 2
        
        self.conv9 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=10, kernel_size=(3,3), stride=1, padding=0,bias=True) )#op = 1 * 1
        

    def forward(self, x):
        x = self.conv3(self.conv2(self.conv1(x)))
        x = self.mp1(x)
        
        x = self.conv6(self.conv5(self.conv4(x)))
        x = self.mp2(x)
        
        x = self.conv9(self.conv8(self.conv7(x)))
        x = x.view(-1,10)
        return F.log_softmax(x,-1)

In [29]:
device = "cuda" if torch.cuda.is_available() else 'cpu'
model = MnistModel().to(device)

In [30]:
summary(model=model,input_size=(1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
       BatchNorm2d-2            [-1, 8, 26, 26]              16
              ReLU-3            [-1, 8, 26, 26]               0
            Conv2d-4           [-1, 16, 24, 24]           1,168
       BatchNorm2d-5           [-1, 16, 24, 24]              32
              ReLU-6           [-1, 16, 24, 24]               0
            Conv2d-7           [-1, 16, 22, 22]           2,320
       BatchNorm2d-8           [-1, 16, 22, 22]              32
              ReLU-9           [-1, 16, 22, 22]               0
        MaxPool2d-10           [-1, 16, 11, 11]               0
           Conv2d-11           [-1, 32, 11, 11]           4,640
      BatchNorm2d-12           [-1, 32, 11, 11]              64
             ReLU-13           [-1, 32, 11, 11]               0
           Conv2d-14             [-1, 6

In [31]:
df = receptive_field(model,28)

=======================================Reciptive Field Calculator========================================
|    | Kernel_size   | Padding   |   Stride | Input_Img_size   | Output_Img_size   | Receptive_field   |
|---:|:--------------|:----------|---------:|:-----------------|:------------------|:------------------|
|  0 | 3*3           | NO        |        1 | 28*28            | 26*26             | 3*3               |
|  1 | 3*3           | NO        |        1 | 26*26            | 24*24             | 5*5               |
|  2 | 3*3           | NO        |        1 | 24*24            | 22*22             | 7*7               |
|  3 | 2*2           | NO        |        2 | 22*22            | 11*11             | 8*8               |
|  4 | 3*3           | 1         |        1 | 11*11            | 11*11             | 12*12             |
|  5 | 3*3           | NO        |        1 | 11*11            | 9*9               | 16*16             |
|  6 | 3*3           | NO        |        1 | 9*9     

In [32]:
class Agent:
    def __init__(self,model,optimizer):
        self.model = model
        self.optimizer = optimizer
    
    def update(self,target_pred,target):
        self.optimizer.zero_grad()
        loss = F.cross_entropy(target_pred,target)
        loss.backward()
        self.optimizer.step()
        return loss

In [33]:
# Initialize the model, optimizer, and criterion
model = MnistModel().to(device)
optimizer = optim.SGD(model.parameters(),lr = 0.01,momentum=0,dampening=0,weight_decay=0)

In [34]:
agent = Agent(model, optimizer)

In [35]:

def trainning(model,device,train_data,optimizer,epochs):
    total_loss = 0
    correct = 0
    processed = 0

    model.train()
    pbar = tqdm(train_data,colour = '\x1b[34m')
    
    for index_id,(data,target) in enumerate(pbar):
        data,target = data.to(device),target.to(device)
        output = model(data)
        loss = agent.update(output,target)
        total_loss+=loss
        correct += output.argmax(dim = 1).eq(target).sum().item()
        processed += len(data)
        pbar.set_description(f"Train ==> Epochs: {epochs} Batch:  {index_id} loss: {loss} Accuracy: { correct/processed *100 :.2f}% ")

    acc = correct /processed
    total_loss = total_loss.item()/processed
    return total_loss, acc

def testing(model,device,test_data,optimizer,epochs):
    test_loss = 0
    correct = 0
    processed = 0
    
    pbar= tqdm(test_data)
    
    model.eval()
    with torch.no_grad():
        for id_x,(data,target) in enumerate(pbar):
            data,target = data.to(device),target.to(device)

            output = model(data)
            test_loss += F.cross_entropy(output,target,reduce='sum').item()
            pred  = output.argmax(dim =1,keepdim = True)

            correct += pred.eq(target.view_as(pred)).sum().item()
            processed += len(data)
            
            pbar.set_description(f"Test ==> Epochs: {epochs} Batch:  {id_x} loss: {test_loss/processed} Accuracy: { correct / processed *100 :.2f}% ")
        
    acc = correct / processed
    test_loss /= processed
    return test_loss,acc,pred,target

In [36]:
train_losses=[]
train_accuracy = []
test_losses = []
test_accuracy = []

for EPOCHS in range(10):
    train_loss, train_acc = trainning(model,device,mnist_train,optimizer,EPOCHS)
    train_losses.append(train_loss)
    train_accuracy.append(train_acc)

    test_loss,test_acc,test_pred,test_target = testing(model,device,mnist_test,optimizer,EPOCHS)

    test_accuracy.append(test_acc)
    test_losses.append(test_loss)

    # Calculate the reward
    reward = test_pred.eq(test_target).float().mean()
    # Update the model
    clean_gpu()
    print(f'Epoch: {EPOCHS},train_losses:{train_losses[-1]}, test_losses:{test_losses[-1]},Reward: {reward:.3f}')
    print ( "================================================================================" *2)

Train ==> Epochs: 0 Batch:  468 loss: 0.06841827183961868 Accuracy: 95.03% : 100
Test ==> Epochs: 0 Batch:  78 loss: 0.0005501965327188372 Accuracy: 98.01% : 100


Epoch: 0,train_losses:0.0015599090576171874, test_losses:0.0005501965327188372,Reward: 0.152


Train ==> Epochs: 1 Batch:  468 loss: 0.1374596506357193 Accuracy: 98.49% : 100%
Test ==> Epochs: 1 Batch:  78 loss: 0.0003803044135682285 Accuracy: 98.60% : 100


Epoch: 1,train_losses:0.0004318549474080404, test_losses:0.0003803044135682285,Reward: 0.125


Train ==> Epochs: 2 Batch:  468 loss: 0.03598952665925026 Accuracy: 99.00% : 100
Test ==> Epochs: 2 Batch:  78 loss: 0.0002990375589346513 Accuracy: 98.93% : 100


...

Epoch: 7,train_losses:8.968451817830403e-05, test_losses:0.00019299411419779062,Reward: 0.141


Train ==> Epochs: 8 Batch:  468 loss: 0.016139620915055275 Accuracy: 99.83% : 10
Test ==> Epochs: 8 Batch:  78 loss: 0.00020224455051939003 Accuracy: 99.10% : 10


Epoch: 8,train_losses:7.270353635152182e-05, test_losses:0.00020224455051939003,Reward: 0.188


Train ==> Epochs: 9 Batch:  468 loss: 0.008511875756084919 Accuracy: 99.88% : 10
Test ==> Epochs: 9 Batch:  78 loss: 0.00019493792654247955 Accuracy: 99.24% : 10


Epoch: 9,train_losses:6.083915630976359e-05, test_losses:0.00019493792654247955,Reward: 0.141
